# Gateway to Research data processing

This notebook documents some exploratory processing of the Gateway to Research dataset with information about research-council and Innovate UK funded projects in the UK. 

In it, we load the project data and enrich it with information about its funding, topics and outputs.

The output is an interim dataset for further enrichment for example with industries, disciplines etc.

### 0. Preamble

#### Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Functions

def missing_count(df,ax):
    '''
    Creates a barchart with share of missing values by variable in the data
    
    '''
    
    df.apply(lambda x: pd.isnull(x),axis=1).mean().sort_values(ascending=False).plot.bar(ax=ax,color='blue')
    
def get_year(date):
    '''
    Extracts years from GTR data. These are strings so we just get the first number
    '''
    
    if pd.isnull(date)==False:
        y = date.split('-')[0]
        return(int(y))
    
    else:
        return(np.nan)



### 1. Load GtR data

Load the data

In [ ]:
my_tables = ['_projects','_organisations','_products','_intellectualproperties','_publications',
             'technicalproducts','_researchdatabaseandmodels',
             '_spinouts','_link_table']

Automatically identify the latest version of the data

In [ ]:
#This gets the maximum date in the gtr directory
latest_data = sorted(
    [(x,datetime.datetime.strptime(x,'%Y-%m-%d')) for x in os.listdir('../data/raw/gtr') if x!='gtr_projects.csv'],key=lambda x:x[1])[-1][0]

In [ ]:
gtr_path= f'../data/raw/gtr/{latest_data}/'


all_tables = os.listdir(gtr_path)

In [ ]:
#A dict where every item is a df with the key = table name. We do a bit of string manipulation to remove dates etc.
my_data = {file.split('_')[-1][:-4]:pd.read_csv(gtr_path+file).iloc[:,1:] for file in all_tables if any(x in file for x in my_tables)}

In [ ]:
my_data.keys()

Load the linking file here because we will use it throughout

In [ ]:
link = my_data['table']

link.head()

### Projects

In [ ]:
projects = my_data['projects']

projects.head()

In [ ]:
projects.shape

len(set(projects['id']))

Some rapid observations:

* what does start mean?
* What does created mean?
* abstractText seems to be the main textual description

In [ ]:
fig,ax = plt.subplots()

missing_count(projects,ax)

ax.set_title('Missing values in Project GTR data')

In [ ]:
projects['year_created'] = projects['created'].apply(lambda x: get_year(x))

projects['year_created'].value_counts()

Created is a data collection variable

In [ ]:
projects['year_started'] = projects['start'].apply(lambda x: get_year(x))

projects['year_started'].value_counts().loc[np.arange(2006,2020)].plot.bar(color='blue',title='Started year')

Not sure about what `year_stared` means but it doesn't cover many records

In [ ]:
projects['leadFunder'].value_counts().plot.bar(color='blue',title='funder_distribution')

EPSRC and Innovate UK are the most active organisations in the data

In [ ]:
projects['grantCategory'].value_counts(normalize=True)[:10].plot.bar(color='blue',title='project types')

Research grants are the main category with almost half of the observations

##### Does the fund data contain the actual start and end date for the project?

In [ ]:
fund = pd.read_csv(gtr_path+'/gtr_funds.csv').iloc[:,1:]

In [ ]:
fund.shape

In [ ]:
fund.sort_values('amount')

Why are there many more funds than projects?

In [ ]:
fund['year'] = fund['start'].apply(lambda x: get_year(x))

fund['year'].value_counts().loc[np.arange(2000,2020)].plot.bar(color='blue',title='Start date (funded data)')

The dataset begins in 2016. What is the bump in 2017?

In [ ]:
pd.crosstab(fund['category'],fund['year'])[np.arange(2006,2020)].T.plot.bar(stacked=True,title='funding by category')

Most of the funding levels in the data reflect income

#### Merge funding with projects to get dates and funding by project

`projects_f = projects merged w/ funding`

In [ ]:
#Merge everything. It works

projects_f = pd.merge(
    pd.merge(fund,link,left_on='id',right_on='id'),
    projects,
    left_on='project_id',right_on='id')

Lots of guff there. I should tidy it up later

In [ ]:
#Why has the number of projects increased so much?
len(projects_f)

In [ ]:
#Drop duplicated project_ids - for some reason we have duplicate entries with different ids in the gtr_funds table

projects_f_no_dupes = projects_f.drop_duplicates('project_id')

#### Let's also check the Topics: does this refer to project keywords?

In [ ]:
topics = pd.read_csv(gtr_path+'/gtr_topic.csv').iloc[:,1:]

topics.head()

In [ ]:
topics['topic_type'].value_counts()

Yes. What are their keys in the linked_table?

In [ ]:
link['table_name'].value_counts()

We can loop over the projects and allocate them topics

Create a research activity lookup and a research topic lookup.

In [ ]:
res_activity_lookup,res_topic_lookup = [{this_id:text for this_id,text in zip(topics.loc[topics['topic_type']==topic_type,'id'],
                                                                             topics.loc[topics['topic_type']==topic_type,'text'])} for
                                        topic_type in ['researchActivity','researchTopic']]
                                                                              

In [ ]:
list(res_activity_lookup.values())[:10]

`researchActivity` is a medical set of subjects

In [ ]:
list(res_topic_lookup.values())[:10]

`researchTopics` is generic

### Organisations


In [ ]:
orgs = my_data['organisations']

orgs.head()

In [ ]:
orgs.shape

This will be interesting to look at collaboration networks between organisations.

Note that this is likely to over-estimate collaboration given that departments (which might be disconnected) will be subsumed under organisations. Maybe we could use the persons data to unpick that?

### Outputs

Brief exploration of the data and standardisation with a single schema (for merging with projects)

#### Products

In [ ]:
products = my_data['products']

In [ ]:
products.head()

In [ ]:
products.shape

They seem to be primarily medical products

In [ ]:
products.stage.value_counts()

In [ ]:
products.type.value_counts()

Yes - this is clearly a medical database. We might use it for our two missions given their health focus

#### Intellectual Property

In [ ]:
ip = my_data['intellectualproperties']

ip.head()

In [ ]:
ip.shape

In [ ]:
ip.protection.value_counts().plot.bar(color='blue',title='ip')

Around 5000 patents and a bunch of random labels. We can use it perhaps to look at field maturity

#### Technical products

In [ ]:
technical = my_data['softwareandtechnicalproducts']

technical.head()

In [ ]:
technical.shape

In [ ]:
technical['type'].value_counts().plot.bar(color='blue',title='Technical outputs')

Perhaps we could create a 'practical output' dummy for the field...

Or query the application databases with project names once we have identified them?

In [ ]:
np.sum(['github' in x for x in technical.supportingUrl if pd.isnull(x)==False])

2582 projects in GitHUb

#### Spinouts

In [ ]:
spinouts = my_data['spinouts']

spinouts.head()

In [ ]:
spinouts.yearEstablished.value_counts().loc[np.arange(2006,2020)].plot.bar(color='blue',title='spinout_year')

#### Publications

In [ ]:
pubs = my_data['publications']

pubs.head()

In [ ]:
#Get missing values for a random sample (10%) of the publications

fig,ax = plt.subplots()

pub_sample = pubs.loc[list(np.random.randint(0,len(pubs),70000)),:]

missing_count(pub_sample,ax)


80% have DOI

In [ ]:
pubs.type.value_counts().plot.bar(color='blue',title='Type of publication')

In [ ]:
pubs['year'] = pubs['datePublished'].apply(get_year)

In [ ]:
pubs['year'].value_counts().loc[np.arange(2006,2020)].plot.bar(color='blue',title='publication years')

In [ ]:
pubs['journalTitle'].value_counts()[:10]

### Databases

In [ ]:
my_data.keys()

In [ ]:
databases = my_data['researchdatabaseandmodels']

databases.shape

In [ ]:
databases.head()

In [ ]:
databases['type'].value_counts().plot.bar(color='blue')

Mostly databases - yay

In [ ]:
pd.crosstab(databases['yearFirstProvided'],databases['type']).plot.bar(stacked=True)

### Discipline coverage check

Here we run outputs vs projects to get who were the funders. We want to check how much cross-organisation coverage there is in the data

We also identify projects that appear in different 'impact' databases

In [ ]:
#Containers for the data
impactful_projects= []

impact_funders = [projects['leadFunder'].value_counts()]

impact_names = ['prods','ip','tech','spin','pubs','databases']

for name,data in zip(impact_names,[products,ip,technical,spinouts,pubs,databases]):
    
    #Merges outputs and projects via the product file
    
    merged = pd.merge(
        pd.merge(data,link,left_on='id',right_on='id'),
        projects,left_on='project_id',right_on='id')
    
    #number of times that a project appears in an output df
    project_counts = merged['project_id'].value_counts()
    project_counts.name = name
    
    #Put it with the featured projects
    impactful_projects.append(project_counts)
    
    
    #Funder impact by project
    funder_freqs = merged['leadFunder'].value_counts()
    funder_freqs.name = name
    
    #Merges with the outpi
    impact_funders.append(funder_freqs)
    

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))

pd.concat(impact_funders,axis=1,sort=False).apply(lambda x: x/x.sum(),axis=0).T.plot.bar(stacked=True,ax=ax,width=0.8)

ax.legend(bbox_to_anchor=(1,1))

Some observations:
* No output data for innovate UK
* MRC over-represented in all outputs (not bad for the mission pilots)
* STFC over-represented in publications(physics)

In [ ]:
# And the impactful projects

#Concatenate the previous outputs
project_impacts = pd.concat(impactful_projects,axis=1,sort=True).fillna(0).reset_index(drop=False)

In [ ]:
#Concatenate with the projects file

projects_imp = pd.merge(projects_f_no_dupes,project_impacts,left_on='project_id',right_on='index',how='left')

#projects_imp = pd.concat([projects_f.set_index('project_id'),project_impacts],axis=1)

projects_imp.shape

In [ ]:
len(set(projects_imp.project_id))

Fillnas with 0s. 
Should drop innovate from follow-on analyses here given that they don't seem to be tracking the impact of their projects in the same way as research councils.

In [ ]:
projects_imp[impact_names] = projects_imp[impact_names].fillna(0)

In [ ]:
import seaborn as sns

In [ ]:
#What is the correlation between different types of outputs...and level of funding?

projects_imp[impact_names+['amount']].corr()

Lots of potential confounders here - types of projects, disciplines...

#### Some observations for next steps

* Analysis focusing on 2006-2018
* We need to integrate the topic data (new link file)?
* Analyses of TRL/output by field will need to consider differences between disciplines in their outlets. To which extent are the disciplines in a mission field more or less productive than their constituent fields?
* ...



## 2. Process data

* Classify projects into disciplines.
  * I need the project topics for this labelling
* Any TRL tags would go in here

Add research activities and topics to the projects data

In [ ]:
projects_imp.set_index('project_id',inplace=True)

In [ ]:
topic_lookup = link.loc[link['table_name']=='gtr_topic',:]

#We simply group by projects and run the names vs the research activity and topic lookup

#Group
res_topics_by_project = topic_lookup.groupby('project_id')

#Extract lookups
activity_project_lookup,topic_project_lookup = [res_topics_by_project['id'].apply(
    lambda x: [lookup[el] for el in [t for t in x] if el in lookup.keys()]).to_dict() for lookup in [res_activity_lookup,res_topic_lookup]]

In [ ]:
#Add topics

projects_imp['research_topics'],projects_imp['research_activities']= [[
    lookup[x] if x in lookup.keys() else [] for x in projects_imp.index] for lookup in [
    topic_project_lookup,activity_project_lookup]]

In [ ]:
projects_imp['has_topic'],projects_imp['has_activities'] = [[len(x)>0 for x in projects_imp[var]] for var in 
                                                            ['research_topics','research_activities']]
                                                            

In [ ]:
pd.crosstab(projects_imp.loc[projects_imp.grantCategory=='Research Grant','year'],
            projects_imp.loc[projects_imp.grantCategory=='Research Grant','has_topic'],normalize=0).plot.bar(
    stacked=True,title='Share of Research Grants with subject data')


Most projects have subjects

In [ ]:
fig,ax = plt.subplots(figsize=(10,5),ncols=2,sharey=True)

pd.crosstab(projects_imp['leadFunder'],projects_imp['has_topic'],normalize=0).plot.barh(stacked=True,ax=ax[0],legend=False)
pd.crosstab(projects_imp['leadFunder'],projects_imp['has_activities'],normalize=0).plot.barh(stacked=True,ax=ax[1])

Almost no topics in MRC - because they label their projects with activities.

For now we will use an older dataset with better topic coverage for labelling (see notebook `02_jmg_discipline_modelling`)

While exploring the data elsewhere I found that most of the labels are 'unclassified'. Let's check the incidence of this.

In [ ]:
#We look for unclassified labels in the research topics
projects_imp['has_uncl'] = ['Unclassified' in top for top in projects_imp['research_topics']]

pd.crosstab(projects_imp['year'],projects_imp['has_uncl'],normalize=0)[True].plot.bar(color='blue',figsize=(10,5),
                                                                                     title='Label = Unclassified')

In [ ]:
#What is the distribution of unclassified labels by funder

pd.crosstab(projects_imp['leadFunder'],projects_imp['has_uncl'],normalize=0).plot.bar()

In [ ]:
pd.crosstab(projects_imp['grantCategory'],projects_imp['has_uncl'],normalize=0).plot.bar()

### Discipline classifier

Save data to train the model in the `02_jmg`... notebook

In [ ]:
gtr_for_pred = projects_imp.dropna(axis=0,subset=['abstractText'])

gtr_for_pred.to_csv(f'../data/processed/{today_str}_gtr_for_prediction.csv',index_label=False)

Load data after predicting

In [ ]:
gtr_predicted = pd.read_csv('../data/processed/13_6_2019_gtr_labelled.csv',index_col=None,compression='zip').iloc[:,1:]

gtr_predicted.columns